# Power Spectrum 
## Calculate the power spectrum using a 3D grid
Generate a the density field of the galaxy positions using a 3D grid
Using the 3D grid, then calculate the power spectrum using the powerbox package

In [1]:
# Power Spectrum plot
import matplotlib.pyplot as plt
import numpy as np
import powerbox as pbox
from Functions import *
from mpl_toolkits import mplot3d
from powerbox import get_power
from scipy import stats
import statistics
import pandas as pd

# Redshift = 0 
#get gals, snaps, sims_props
snapshot_used_0 = 250 # z = 0
gals, sim_props, snaplist = get_gal_catalogue(snapshot_used_0)
#ngals = len(gals)
#print(ngals)
#gals.columns #returns all avaiable properties of galxies
#gals[gals.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)]# remove infs
#gals.dropna()# drop nan values

# See other available Snaps and Redshifts
reds = snaplist[1]
snaps = snaplist[0]
print(reds)
print(snaps)

# using positions [x,y,x]
pos_x = gals["Pos_0"]
pos_y = gals["Pos_1"]
pos_z = gals["Pos_2"]
Pos = np.array([pos_x, pos_y, pos_z])

snaplist_index_used_0 = list(snaplist[0]).index(snapshot_used_0)# convert to list and find the index of the snap we are using
redshift_used_0 = snaplist[1][snaplist_index_used_0] # Get the redshift for corresponding snapshot
redshift_used_0 = str(round(redshift_used_0, 1)) # round float to nearest whole int
gals.columns #returns all avaiable properties of galxies

[4.9999988  4.01575197 2.99733298 2.00100053 1.50871276 0.99933302
 0.50100094 0.        ]
[100 115 134 158 173 192 216 250]


Index(['HaloID', 'ID', 'Type', 'CentralGal', 'GhostFlag', 'Len', 'MaxLen',
       'Spin', 'Mvir', 'Rvir', 'Vvir', 'Vmax', 'FOFMvir', 'HotGas',
       'MetalsHotGas', 'ColdGas', 'MetalsColdGas', 'H2Frac', 'H2Mass',
       'HIMass', 'Mcool', 'DiskScaleLength', 'StellarMass', 'GrossStellarMass',
       'MetalsStellarMass', 'Sfr', 'EjectedGas', 'MetalsEjectedGas',
       'BlackHoleMass', 'Rcool', 'Cos_Inc', 'MergTime', 'MergerStartRadius',
       'BaryonFracModifier', 'FOFMvirModifier', 'MvirCrit', 'MergerBurstMass',
       'MWMSA', 'Fesc', 'FescWeightedGSM', 'FescBH', 'BHemissivity',
       'EffectiveBHM', 'BlackHoleAccretedHotMass', 'BlackHoleAccretedColdMass',
       'dt', 'Pos_0', 'Pos_1', 'Pos_2', 'Vel_0', 'Vel_1', 'Vel_2',
       'NewStars_0', 'NewStars_1', 'NewStars_2', 'NewStars_3', 'NewStars_4'],
      dtype='object')

In [2]:
sim_props

{'BaryonFrac': 0.15714,
 'BaryonFracModifier': '',
 'BetaBands': '',
 'BirthCloudLifetime': 0.0,
 'BlackHoleGrowthRate': 0.05,
 'BlackHoleMassLimitReion': -1.0,
 'BlackHoleSeed': 1e-07,
 'BoxSize': 178.57142857142858,
 'CatalogFilePrefix': 'subfind',
 'CoolingFuncsDir': '/home/smutch/models/meraxes/src/../input/cooling_functions',
 'DefaultsFile': '/home/smutch/models/meraxes/src/../input/Tiamat125-HR.par',
 'EddingtonRatio': 1.0,
 'EndRedshiftLightcone': 6.0,
 'EndSnapshotLightcone': 0,
 'EscapeFracBHNorm': 1.0,
 'EscapeFracBHScaling': 0.0,
 'EscapeFracDependency': 1,
 'EscapeFracNorm': 0.06,
 'EscapeFracPropScaling': 0.5,
 'EscapeFracRedshiftScaling': 0.5,
 'FileNameGalaxies': 'meraxes',
 'FlagIgnoreProgIndex': 0,
 'FlagInteractive': 0,
 'FlagMCMC': 0,
 'FlagSubhaloVirialProps': 0,
 'Flag_BHFeedback': 1,
 'Flag_Compute21cmBrightTemp': 1,
 'Flag_ComputePS': 1,
 'Flag_ConstructLightcone': 1,
 'Flag_FixDiskRadiusOnInfall': 0,
 'Flag_FixVmaxOnInfall': 0,
 'Flag_IRA': 0,
 'Flag_IncludePec

## Create 3D grid of galaxy positions
### 1. Calculate the edges of the grid
### 2. Calculate the deltax
    delta x = (Pos/edges)
### 3. Get the 3D grid
    using histogram 3D

In [11]:
# getting bits ready for 3D grid 
boxsize = 178.57142857142858 #rounded from 178.57142857142858
dim = len(Pos) # =3
boxlength = [boxsize] * dim
N = pos_x.shape # = 1648367
#N = 100
V = np.product(boxlength)
print('dim : ', dim)
print('boxlength : ', boxlength)
print('N : ', N)
print('V : ', V)
#N = [N] * dim # to make iterable
print('N : ', N)
print('Pos : ', Pos)
#Pos = np.squeeze(Pos)
#print('Pos : ', Pos)


dim :  3
boxlength :  [178.57142857142858, 178.57142857142858, 178.57142857142858]
N :  (1648367,)
V :  5694241.982507289
N :  (1648367,)
Pos :  [[132.21861  135.23091   29.453735 ... 161.74535  177.04092  172.94582 ]
 [ 95.35761   80.398964  51.996937 ... 104.77806  161.79611  178.1844  ]
 [104.57693  126.85246  113.23257  ... 121.301575 160.85045  105.88463 ]]


In [18]:
# --- histogram stuff ---- #
edges = [np.linspace(0, L, n) for L, n in zip(boxlength, N)]
print('edges[0].shape : ', edges[0].shape)
print('edges : ', edges)
#binsx = edges[0].shape
#edges = np.squeeze(edges)
print('edges : ', edges)
print('edges shape : ', len(edges))
print('Pos shape : ', Pos.shape)


edges[0].shape :  (1648367,)
edges :  [array([0.00000000e+00, 1.08332390e-04, 2.16664780e-04, ...,
       1.78571212e+02, 1.78571320e+02, 1.78571429e+02])]
edges :  [array([0.00000000e+00, 1.08332390e-04, 2.16664780e-04, ...,
       1.78571212e+02, 1.78571320e+02, 1.78571429e+02])]
edges shape :  1
Pos shape :  (3, 1648367)


In [20]:

deltax = np.histogramdd(Pos, bins=edges, weights=None)[0].astype("float")

ValueError: The dimension of bins must be equal to the dimension of the  sample x.

## Do calculations (mean and total)
Calculate the average number of galaxies (mean)
And the num of galaxies (which should jsut be the total number in the sample 1648367.0)

## Calculate the density and the PS

In [ ]:
#p_k_field, bins_field = get_power(deltax, binsx)
#p_k_field
#bins_field

# PLOT